# Задание 
### На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [7]:
import pandas as pd
from string import punctuation
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten, add
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from stop_words import get_stop_words

In [3]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_val = pd.read_csv("./data/val.csv")

In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [9]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [10]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [11]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [21]:
from keras.callbacks import EarlyStopping  
early_stopping=EarlyStopping(monitor='val_loss') 

### Свёрточная сеть

In [31]:
model_cnn = Sequential()

model_cnn.add(Embedding(input_dim=word_count, 
                        input_length=training_length, 
                        output_dim=30))

model_cnn.add(Conv1D(30, 3, activation='relu'))
model_cnn.add(GlobalMaxPool1D())
model_cnn.add(Flatten())

model_cnn.add(Dense(10, activation='relu'))
model_cnn.add(Dropout(0.5))

model_cnn.add(Dense(1, activation='sigmoid'))

In [32]:
model_cnn.compile(optimizer='adam',
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

In [33]:
model_cnn.fit(X_train, y_train,
              batch_size=512,
              epochs=10,
              verbose=1,
              validation_split=0.1,
              callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 16s 51ms/step - loss: 0.6058 - accuracy: 0.6568 - val_loss: 0.5118 - val_accuracy: 0.7462
Epoch 2/10
319/319 [==============================] - 17s 53ms/step - loss: 0.4173 - accuracy: 0.8173 - val_loss: 0.5029 - val_accuracy: 0.7510
Epoch 3/10
319/319 [==============================] - 16s 51ms/step - loss: 0.2459 - accuracy: 0.9009 - val_loss: 0.6029 - val_accuracy: 0.7444


In [34]:
score = model_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 2ms/step - loss: 0.6667 - accuracy: 0.7306
Test score: 0.6666739583015442
Test accuracy: 0.7306352853775024


### RNN сеть

In [35]:
model_rnn = Sequential()

model_rnn.add(Embedding(input_dim=word_count, 
                        output_dim=30, 
                        input_length=training_length))

model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(64, activation='relu'))
model_rnn.add(Dropout(0.5))

model_rnn.add(Dense(1, activation='sigmoid'))

In [36]:
model_rnn.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

In [37]:
model_rnn.fit(X_train, y_train,
              batch_size=512,
              epochs=10,
              verbose=1,
              validation_split=0.1,
              callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 24s 76ms/step - loss: 0.5586 - accuracy: 0.7008 - val_loss: 0.4908 - val_accuracy: 0.7560
Epoch 2/10
319/319 [==============================] - 26s 81ms/step - loss: 0.2868 - accuracy: 0.8837 - val_loss: 0.5703 - val_accuracy: 0.7426


In [38]:
score = model_rnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.5967 - accuracy: 0.7293
Test score: 0.5966779589653015
Test accuracy: 0.7293127179145813


### LSTM сеть

In [39]:
model_lstm = Sequential()

model_lstm.add(Embedding(input_dim=word_count, 
                         output_dim=30, 
                         input_length=training_length))

model_lstm.add(LSTM(64, recurrent_dropout=0.2))

model_lstm.add(Dense(64, activation='relu'))

model_lstm.add(Dropout(0.5))

model_lstm.add(Dense(1, activation='sigmoid'))

In [40]:
model_lstm.compile(optimizer='adam', 
                   loss='binary_crossentropy', 
                   metrics=['accuracy'])

In [41]:
model_lstm.fit(X_train, y_train,
               batch_size=512,
               epochs=10,
               verbose=1,
               validation_split=0.1,
               callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 57s 178ms/step - loss: 0.5562 - accuracy: 0.7059 - val_loss: 0.4899 - val_accuracy: 0.7564
Epoch 2/10
319/319 [==============================] - 65s 204ms/step - loss: 0.3326 - accuracy: 0.8623 - val_loss: 0.5401 - val_accuracy: 0.7498


In [42]:
score = model_lstm.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 36ms/step - loss: 0.5852 - accuracy: 0.7380
Test score: 0.585203230381012
Test accuracy: 0.7379975914955139


### GRU сеть

In [43]:
model_gru = Sequential()

model_gru.add(Embedding(input_dim=word_count, 
                        input_length=training_length, 
                        output_dim=30))

model_gru.add(GRU(64, recurrent_dropout=0.2))

model_gru.add(Dense(64, activation='relu'))

model_gru.add(Dropout(0.5))

model_gru.add(Dense(1, activation='sigmoid'))

In [44]:
model_gru.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

In [45]:
model_gru.fit(X_train, y_train,
              batch_size=512,
              epochs=10,
              verbose=1,
              validation_split=0.1,
              callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 53s 166ms/step - loss: 0.5554 - accuracy: 0.7032 - val_loss: 0.4939 - val_accuracy: 0.7541
Epoch 2/10
319/319 [==============================] - 54s 169ms/step - loss: 0.3220 - accuracy: 0.8674 - val_loss: 0.5485 - val_accuracy: 0.7495


In [46]:
score = model_gru.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 22ms/step - loss: 0.5946 - accuracy: 0.7372
Test score: 0.5945937633514404
Test accuracy: 0.7372481822967529


### Сеть CNN -> RNN

In [47]:
model_cnn_rnn = Sequential()

model_cnn_rnn.add(Embedding(input_dim=word_count, 
                            input_length=training_length, 
                            output_dim=30))

model_cnn_rnn.add(Conv1D(filters=64, 
                         kernel_size=3, 
                         activation='relu', 
                         padding="same"))

model_cnn_rnn.add(LSTM(64, return_sequences=True))
model_cnn_rnn.add(Flatten())

model_cnn_rnn.add(Dense(64, activation='relu'))

model_cnn_rnn.add(Dropout(0.5))

model_cnn_rnn.add(Dense(1, activation='sigmoid'))

In [48]:
model_cnn_rnn.compile(optimizer='adam', 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])

In [49]:
model_cnn_rnn.fit(X_train, y_train,
                  batch_size=512,
                  epochs=10,
                  verbose=1,
                  validation_split=0.1,
                  callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 44s 137ms/step - loss: 0.5540 - accuracy: 0.7047 - val_loss: 0.4845 - val_accuracy: 0.7597
Epoch 2/10
319/319 [==============================] - 51s 161ms/step - loss: 0.3037 - accuracy: 0.8758 - val_loss: 0.5304 - val_accuracy: 0.7439


In [50]:
score = model_cnn_rnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 52ms/step - loss: 0.5583 - accuracy: 0.7382
Test score: 0.5582707524299622
Test accuracy: 0.7381739616394043


### Сеть RNN -> CNN

In [52]:
model_rnn_cnn = Sequential()

model_rnn_cnn.add(Embedding(input_dim=word_count, 
                            input_length=training_length, 
                            output_dim=30))

model_rnn_cnn.add(LSTM(64, return_sequences=True))

model_rnn_cnn.add(Conv1D(filters=64, 
                         kernel_size=3, 
                         activation='relu', 
                         padding="same"))
model_rnn_cnn.add(Flatten())

model_rnn_cnn.add(Dense(64, activation='relu'))

model_rnn_cnn.add(Dropout(0.5))

model_rnn_cnn.add(Dense(1, activation='sigmoid'))

In [53]:
model_rnn_cnn.compile(optimizer='adam', 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])

In [54]:
model_rnn_cnn.fit(X_train, y_train,
                  batch_size=512,
                  epochs=10,
                  verbose=1,
                  validation_split=0.1,
                  callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 45s 141ms/step - loss: 0.5528 - accuracy: 0.7078 - val_loss: 0.4902 - val_accuracy: 0.7582
Epoch 2/10
319/319 [==============================] - 47s 146ms/step - loss: 0.3096 - accuracy: 0.8717 - val_loss: 0.5507 - val_accuracy: 0.7466


In [55]:
score = model_rnn_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 43ms/step - loss: 0.5705 - accuracy: 0.7394
Test score: 0.5704938769340515
Test accuracy: 0.7394083738327026


Судя по результатам архетектуры различаются незначаительно, однако лучший результат показывает cnn -> rnn сеть